In [4]:
from lightgbm import LGBMRegressor
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, LabelEncoder, RobustScaler, StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.metrics import accuracy_score, f1_score, mean_squared_error
from sklearn.linear_model import Ridge, Lasso, LassoCV, LinearRegression
import time
from datetime import datetime
import matplotlib.pyplot as plt
import os
import numpy.ma as ma
from sklearn import base
import warnings
from bayes_opt import BayesianOptimization
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events
from tqdm import tqdm
warnings.filterwarnings('ignore')

In [5]:
path_train = "data/train_data.csv"
path_test = "data/test_data.csv"
target_ = "contest-tmp2m-14d__tmp2m"

In [6]:
df_train = pd.read_csv(path_train)
df_test = pd.read_csv(path_test)

date_col = "startdate"
target_ = 'contest-tmp2m-14d__tmp2m'

df_train[date_col] = pd.to_datetime(df_train[date_col])
df_test[date_col] = pd.to_datetime(df_test[date_col])

In [7]:
df_train.head()

,index,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,...,wind-vwnd-925-2010-11,wind-vwnd-925-2010-12,wind-vwnd-925-2010-13,wind-vwnd-925-2010-14,wind-vwnd-925-2010-15,wind-vwnd-925-2010-16,wind-vwnd-925-2010-17,wind-vwnd-925-2010-18,wind-vwnd-925-2010-19,wind-vwnd-925-2010-20
0,0,0.0,0.833333,2014-09-01,237.00,29.02,31.64,29.57,30.73,29.71,...,-27.68,-37.21,8.32,9.56,-2.03,48.13,28.09,-13.50,11.90,4.58
1,1,0.0,0.833333,2014-09-02,228.90,29.02,31.64,29.57,30.73,29.71,...,-21.13,-36.57,8.77,21.17,4.44,48.60,27.41,-23.77,15.44,3.42
2,2,0.0,0.833333,2014-09-03,220.69,29.02,31.64,29.57,30.73,29.71,...,-10.72,-34.16,6.99,32.16,5.01,48.53,19.21,-33.16,15.11,4.82
3,3,0.0,0.833333,2014-09-04,225.28,29.02,31.64,29.57,30.73,29.71,...,0.33,-31.04,6.17,39.66,-1.41,50.59,8.29,-37.22,18.24,9.74
4,4,0.0,0.833333,2014-09-05,237.24,29.02,31.64,29.57,30.73,29.71,...,9.83,-31.80,7.47,38.62,-5.21,54.73,-2.58,-42.30,21.91,10.95


# Fill null

In [8]:
def filter_na_cols(df):
    count_na_df = df.isna().sum()
    if count_na_df[count_na_df > 0].tolist():
        return count_na_df[count_na_df > 0]
    else:
        return 'Clean dataset'

col_na = [
    'nmme0-tmp2m-34w__ccsm30',
    'nmme-tmp2m-56w__ccsm3',
    'nmme-prate-34w__ccsm3',
    'nmme0-prate-56w__ccsm30',
    'nmme0-prate-34w__ccsm30',
    'nmme-prate-56w__ccsm3',
    'nmme-tmp2m-34w__ccsm3']

g_means =  ['nmme0-tmp2m-34w__nmme0mean',
 'nmme-tmp2m-56w__nmmemean',
 'nmme-prate-34w__nmmemean',
 'nmme0-prate-56w__nmme0mean',
 'nmme0-prate-34w__nmme0mean',
 'nmme-prate-56w__nmmemean',
 'nmme-tmp2m-34w__nmmemean']


g_1 = ['nmme0-tmp2m-34w__cancm30',
'nmme0-tmp2m-34w__cancm40',
'nmme0-tmp2m-34w__ccsm40',
'nmme0-tmp2m-34w__cfsv20',
'nmme0-tmp2m-34w__gfdlflora0',
'nmme0-tmp2m-34w__gfdlflorb0',
'nmme0-tmp2m-34w__gfdl0',
'nmme0-tmp2m-34w__nasa0']

g_2 = ['nmme-tmp2m-56w__cancm3',
'nmme-tmp2m-56w__cancm4',
'nmme-tmp2m-56w__ccsm4',
'nmme-tmp2m-56w__cfsv2',
'nmme-tmp2m-56w__gfdl',
'nmme-tmp2m-56w__gfdlflora',
'nmme-tmp2m-56w__gfdlflorb',
'nmme-tmp2m-56w__nasa']

g_3 = ['nmme-prate-34w__cancm3',
'nmme-prate-34w__cancm4',
'nmme-prate-34w__ccsm4',
'nmme-prate-34w__cfsv2',
'nmme-prate-34w__gfdl',
'nmme-prate-34w__gfdlflora',
'nmme-prate-34w__gfdlflorb',
'nmme-prate-34w__nasa']

g_4 = [ 'nmme0-prate-56w__cancm30',
'nmme0-prate-56w__cancm40',
'nmme0-prate-56w__ccsm40',
'nmme0-prate-56w__cfsv20',
'nmme0-prate-56w__gfdlflora0',
'nmme0-prate-56w__gfdlflorb0',
'nmme0-prate-56w__gfdl0',
'nmme0-prate-56w__nasa0']

g_5 = ['nmme0-prate-34w__cancm30',
'nmme0-prate-34w__cancm40',
'nmme0-prate-34w__ccsm40',
'nmme0-prate-34w__cfsv20',
'nmme0-prate-34w__gfdlflora0',
'nmme0-prate-34w__gfdlflorb0',
'nmme0-prate-34w__gfdl0',
'nmme0-prate-34w__nasa0']

g_6 = ['nmme-prate-56w__cancm3',
'nmme-prate-56w__cancm4',
'nmme-prate-56w__ccsm4',
'nmme-prate-56w__cfsv2',
'nmme-prate-56w__gfdl',
'nmme-prate-56w__gfdlflora',
'nmme-prate-56w__gfdlflorb',
'nmme-prate-56w__nasa']

g_7 = ['nmme-tmp2m-34w__cancm3',
'nmme-tmp2m-34w__cancm4',
'nmme-tmp2m-34w__ccsm4',
'nmme-tmp2m-34w__cfsv2',
'nmme-tmp2m-34w__gfdl',
'nmme-tmp2m-34w__gfdlflora',
'nmme-tmp2m-34w__gfdlflorb',
'nmme-tmp2m-34w__nasa']

gs = [g_1, g_2, g_3, g_4, g_5, g_6, g_7]

zip_cols = zip(col_na, gs, g_means)
for c, g, m in tqdm(zip_cols):
    df_train[c] = (df_train[m]*9) - df_train[g].sum(1)

7it [00:02,  2.35it/s]


In [9]:
df_train.drop(columns='ccsm30', inplace=True)
df_test.drop(columns='ccsm30', inplace=True)

# Feature engineering

In [10]:
drop_col = pd.read_csv("data/correlations_with_target_greater_0.7.csv")
drop_col = drop_col["col"].values
drop_col = [each for each in drop_col if "contest" not in each and "wind" not in each]

## Bining

In [11]:
bin_col = ['nmme-tmp2m-34w__cfsv2', 'nmme-tmp2m-56w__gfdlflora', 'nmme-tmp2m-56w__nasa', 'nmme0-tmp2m-34w__cancm30', 'nmme-tmp2m-56w__gfdl', 'nmme0mean']

list_new_col = []

def bin_feature_tpm2m(x):
    if x < -5:
        return 'A'
    elif x < 0:
        return 'B'
    elif x < 5:
        return 'C'
    elif x < 10:
        return 'D'
    elif x < 15:
        return 'E'
    elif x < 20:
        return 'F'
    elif x < 25:
        return 'G'
    else:
        return 'H'

for each in bin_col:
    df_train[each + "_bin"] = np.vectorize(bin_feature_tpm2m)(df_train[each])
    df_test[each + "_bin"] = np.vectorize(bin_feature_tpm2m)(df_test[each])
    list_new_col.append(each + "_bin")

In [12]:
def get_idx(lat, lon):
    return str(round(lat, 4)) + "_" + str(round(lon, 4))

df_train['idx'] = np.vectorize(get_idx)(df_train['lat'], df_train['lon'])
df_test['idx'] = np.vectorize(get_idx)(df_test['lat'], df_test['lon'])

In [13]:
col = [each for each in df_train.columns if "contest" in each and each != target_]

for each in col:
    df_train[each + "_lag_1"] = df_train.groupby("idx")[each].shift(1).bfill()
    df_test[each + "_lag_1"] = df_test.groupby("idx")[each].shift(1).bfill()
    df_train[each + "_lag_2"] = df_train.groupby("idx")[each].shift(2).bfill()
    df_test[each + "_lag_2"] = df_test.groupby("idx")[each].shift(2).bfill()
    list_new_col.append(each + "_lag_1")
    list_new_col.append(each + "_lag_2")

In [14]:
col_test = [each for each in df_train.columns if "contest" not in each and ("wind" in each or "sst" in each)]
col_drift = [
    "wind-vwnd-250-2010-2",
    "wind-vwnd-250-2010-16",
    "wind-vwnd-250-2010-19",
    "wind-uwnd-250-2010-9",
    "wind-uwnd-250-2010-11",
    "wind-uwnd-250-2010-17",
    "wind-hgt-850-2010-2",
    "wind-hgt-850-2010-5",
    "wind-hgt-850-2010-7",
    "sst-2010-2",
    "sst-2010-10",
    "wind-hgt-500-2010-5",
    "wind-hgt-500-2010-7",
    "wind-hgt-500-2010-8",
    "wind-uwnd-925-2010-2",
    "wind-uwnd-925-2010-7",
    "wind-uwnd-925-2010-20"
]

dict_col_drift = {each:True for each in col_drift}

for each in col_test:
    if each in dict_col_drift:
        continue
    df_train[each + "_lag_1"] = df_train.groupby("idx")[each].shift(1).bfill()
    df_test[each + "_lag_1"] = df_test.groupby("idx")[each].shift(1).bfill()
    df_train[each + "_diff"] = df_train.groupby("idx")[each].diff().bfill()
    df_test[each + "_diff"] = df_test.groupby("idx")[each].diff().bfill()
    list_new_col.append(each + "_diff")
    list_new_col.append(each + "_lag_1")

In [15]:
def create_mean_group(df, cols_group, cols_mean, col_date="startdate"):
    list_new_col = []
    df[col_date] = df[col_date].astype("str")

    for col_mean in cols_mean:
        for col_group in cols_group:
            new_col = col_mean + "_{}".format(col_group) + "_mean"
            dict_map = df.groupby([col_group, col_date])[col_mean].mean().to_dict()

            def map_dict(gr, date):
                return dict_map[gr, date]

            df[new_col] = np.vectorize(map_dict)(df[col_group], df[col_date])
            list_new_col.append(new_col)

    df[col_date] = pd.to_datetime(df[col_date])

    return df, list_new_col

cols_group = ["lat", "lon", "climateregions__climateregion"]
df_train, list_col = create_mean_group(df_train, cols_group, col)
df_test, _ = create_mean_group(df_test, cols_group, col)
list_new_col = [*list_new_col, *list_col]

In [16]:
def get_name_cols(suffix='', cols=[]):
    name_cols = [col + f'{suffix}' for col in cols if col != 'contest-tmp2m-14d__tmp2m']

    return name_cols

# Add wind features

def feature_engineer(df, cols):
    rolling_mean_contest_cols_3 = df.groupby('idx')[cols].transform(lambda x: x.rolling(window=3, min_periods=2).mean()) \
        .rename(columns=dict(zip(cols, get_name_cols('_rolling_mean_3', cols))))
    rolling_std_contest_cols_3 = df.groupby('idx')[cols].transform(lambda x: x.rolling(window=3, min_periods=2).std()) \
        .rename(columns=dict(zip(cols, get_name_cols('_rolling_std_3', cols))))
    rolling_mean_contest_cols_6 = df.groupby('idx')[cols].transform(lambda x: x.rolling(window=6, min_periods=2).mean()) \
        .rename(columns=dict(zip(cols, get_name_cols('_rolling_mean_6', cols))))
    rolling_std_contest_cols_6 = df.groupby('idx')[cols].transform(lambda x: x.rolling(window=6, min_periods=2).std()) \
        .rename(columns=dict(zip(cols, get_name_cols('_rolling_std_6', cols))))
    diff_contest_cols = df.groupby('idx')[cols].transform(lambda x: x.diff()) \
        .rename(columns=dict(zip(cols, get_name_cols('_diff', cols))))
    exponentially_weighted_average = df.groupby('idx')[cols].transform(lambda x: x.shift(1).ewm(alpha=0.95).mean()) \
        .rename(columns=dict(zip(cols, get_name_cols('_ewa', cols))))

    df = pd.concat([df, rolling_mean_contest_cols_3, rolling_std_contest_cols_3,
                    rolling_mean_contest_cols_6, rolling_std_contest_cols_6, diff_contest_cols,
                    exponentially_weighted_average], axis=1)
    df = df.fillna(method='bfill')

    return df

def feature_engineer_2(df, cols):
    rolling_mean_contest_cols_3 = df.groupby('idx')[cols].transform(lambda x: x.rolling(window=3, min_periods=2).mean()) \
        .rename(columns=dict(zip(cols, get_name_cols('_rolling_mean_3', cols))))
    rolling_std_contest_cols_3 = df.groupby('idx')[cols].transform(lambda x: x.rolling(window=3, min_periods=2).std()) \
        .rename(columns=dict(zip(cols, get_name_cols('_rolling_std_3', cols))))
    # diff_contest_cols = df.groupby('idx')[cols].transform(lambda x: x.diff()) \
    #     .rename(columns=dict(zip(cols, get_name_cols('_diff', cols))))
    exponentially_weighted_average = df.groupby('idx')[cols].transform(lambda x: x.shift(1).ewm(alpha=0.95).mean()) \
        .rename(columns=dict(zip(cols, get_name_cols('_ewa', cols))))

    df = pd.concat([df, rolling_mean_contest_cols_3,
                    rolling_std_contest_cols_3,
                    exponentially_weighted_average],
                   axis=1)
    df = df.fillna(method='bfill')

    return df

imp_contest_cols = [
    'contest-pevpr-sfc-gauss-14d__pevpr',
    'contest-pres-sfc-gauss-14d__pres',
    'contest-prwtr-eatm-14d__prwtr',
    'contest-slp-14d__slp',
    'contest-wind-h10-14d__wind-hgt-10',
    'contest-wind-h100-14d__wind-hgt-100',
    'contest-wind-h500-14d__wind-hgt-500',
    'contest-wind-uwnd-250-14d__wind-uwnd-250',
    'contest-wind-vwnd-925-14d__wind-vwnd-925'
]

drift_nnme_prate_cols = [
    * [col for col in col_test if col not in dict_col_drift],
    * [col for col in df_train.columns if 'nmme-prate' in col or 'nmme0-prate' in col]
]

df_train = feature_engineer(df_train, imp_contest_cols)
df_test = feature_engineer(df_test, imp_contest_cols)

df_train = feature_engineer_2(df_train, drift_nnme_prate_cols)
df_test = feature_engineer_2(df_test, drift_nnme_prate_cols)

In [17]:
col_ = [each for each in df_train.columns if "_rolling" in each or "_ewa" in each]
list_new_col = [*list_new_col, *col_]

In [18]:
def sin_transformer(period):
    return FunctionTransformer(lambda x: np.sin(x / period * 2 * np.pi))

def cos_transformer(period):
    return FunctionTransformer(lambda x: np.cos(x / period * 2 * np.pi))

def handle_idx_date(df, column_date, idx_name):
    def get_idx(lat, lon):
        return str(round(lat, 4)) + "_" + str(round(lon, 4))
    df[idx_name] = np.vectorize(get_idx)(df['lat'], df['lon'])
    if column_date is not None:
        df[column_date] = pd.to_datetime(df[column_date])
        df['day_of_year'] = df[column_date].dt.day_of_year
        df['month'] = df[column_date].dt.month
          # encode the day with a period of 365
        df['day_of_year_sin'] = sin_transformer(365).fit_transform(df['day_of_year'])
        df['day_of_year_cos'] = cos_transformer(365).fit_transform(df['day_of_year'])

        # encode the month with a period of 12
        df['month_sin'] = sin_transformer(12).fit_transform(df['month'])
        df['month_cos'] = cos_transformer(12).fit_transform(df['month'])
    return df

def handle_feature_train_data(df, column_date="startdate", columns_cat = [], idx_name="idx", norm="min_max"):
    df = handle_idx_date(df, column_date, idx_name)
    if column_date is not None:
        df = df.drop(columns = [column_date])
    columns_cat.append(idx_name)
    list_lbEncoder = []

    for each in columns_cat:
        lbE = LabelEncoder().fit(df[each])
        df[each] = lbE.transform(df[each])
        list_lbEncoder.append(lbE)

    # for each in df.columns:
    #     if df[each].isna().sum() != 0:
    #         df[each] = df.groupby(idx_name)[each].transform(lambda x: x.ffill())

    df[columns_cat] = df[columns_cat].astype("category")

    X = None
    if norm == "min_max":
        X = MinMaxScaler().fit_transform(df.values)
    elif norm == "std":
        X = StandardScaler().fit_transform(df.values)

    return df, X, list_lbEncoder

def handle_feature_test_data(df, lbEncoder, column_date="startdate", columns_cat = [], idx_name="idx"):
    df = handle_idx_date(df, column_date, idx_name)
    if column_date is not None:
        df = df.drop(columns = [column_date])
    columns_cat.append(idx_name)

    for index, each in enumerate(columns_cat):
        df[each] = lbEncoder[index].transform(df[each])

    # for each in df.columns:
    #     if df[each].isna().sum() != 0:
    #         df[each] = df.groupby(idx_name)[each].transform(lambda x: x.ffill())

    df[columns_cat] = df[columns_cat].astype("category")
    return df

In [19]:
def reconvert(df):
    if not os.path.exists('delete_later'):
        os.mkdir('delete_later')

    df.to_csv('delete_later/df.csv')
    df = pd.read_csv('delete_later/df.csv')

    return df

In [20]:
def calculate_corr_target(X_train, X_val):
    data_col = []
    data_corr_train = []
    data_corr_val = []

    for col in X_train.columns:
        corr_train = X_train['contest-tmp2m-14d__tmp2m'].corr(X_train[col])
        corr_val = X_val['contest-tmp2m-14d__tmp2m'].corr(X_val[col])

        data_col.append(col)
        data_corr_train.append(corr_train)
        data_corr_val.append(corr_val)

    corr = pd.DataFrame(data={'col': data_col, 'corr_train': data_corr_train, 'corr_val': data_corr_val})

    return corr

In [21]:
corr_df = pd.read_csv("data/correlations_with_target.csv") \
            .drop(columns='Unnamed: 0')
corr_df = corr_df[(corr_df['corr'] >= 0.7)  | (corr_df['corr'] <= -0.7)]
drop_col = corr_df["col"].values
drop_col = [each for each in drop_col if "contest" not in each and "wind" not in each and each != 'ccsm30']

# Model

## LGBM

In [22]:
def bayes_tunning():
    def CB_opt(max_depth, num_leaves, colsample_bytree, reg_alpha,
               reg_lambda, subsample, feature_fraction_bynode):

        lgb = LGBMRegressor(
            max_depth = round(max_depth),
            num_leaves=round(num_leaves),
            colsample_bytree=colsample_bytree,
            reg_alpha=reg_alpha,
            reg_lambda=reg_lambda,
            subsample=subsample,
            feature_fraction_bynode=feature_fraction_bynode,
            verbose=0,
            n_estimators = 7999,
            metric="rmse",
            boosting_type="dart",
            learning_rate=0.05,
            n_jobs=-1,
            extra_trees=True
        )

        lgb.fit(X_train, y_train, eval_metric="rmse",
                categorical_feature=[
                    'climateregions__climateregion', 'idx', 'mjo1d__phase',
                    'nmme-tmp2m-34w__nmmemean_bin', 'nmme-tmp2m-56w__nmmemean_bin',
                    'nmme0-tmp2m-34w__nmme0mean_bin', 'nmme0mean_bin'
                ]
                )
        ypred_valid = lgb.predict(X_valid)

        return 1 / mean_squared_error(y_valid, ypred_valid, squared=False)

    pbounds = {
        'max_depth': (8, 17),
        'num_leaves': (15, 35),
        'colsample_bytree': (0.5, 0.8),
        'reg_alpha': (0.1, 3),
        'reg_lambda': (0.1, 3),
        # 'min_split_gain': (0.001, 0.3),
        'subsample': (0.5, 0.85),
        'feature_fraction_bynode': (0.6, 0.85)
    }

    if not os.path.exists('Params_tunning/Log'):
        os.makedirs('Params_tunning/Log')

    optimizer = BayesianOptimization(f = CB_opt, pbounds = pbounds, verbose = 2, random_state = 209)
    logger = JSONLogger(path="Params_tunning/Log/logs_{}.json".format(i))
    optimizer.subscribe(Events.OPTIMIZATION_STEP, logger)
    optimizer.maximize(init_points = 5, n_iter = 15)

    print(optimizer.max['target'])

    max_bo_params = optimizer.max['params']

    return max_bo_params

In [23]:
def get_bo_result(max_bo_params):
    opt_params = {
        'max_depth': round(max_bo_params['max_depth']),
        'num_leaves': round(max_bo_params['num_leaves']),
        'colsample_bytree': max_bo_params['colsample_bytree'],
        'reg_alpha': max_bo_params['reg_alpha'],
        'reg_lambda': max_bo_params['reg_lambda'],
        'subsample': max_bo_params['subsample'],
        'feature_fraction_bynode': max_bo_params['feature_fraction_bynode'],
        'verbose': 0,
        'n_estimators': 7999,
        'metric': 'rmse',
        'boosting_type': 'dart',
        'learning_rate': 0.05,
        'n_jobs': -1
    }

    return opt_params

def get_best_params(path):
    tmp = pd.read_json(path, lines=True)
    max_t = 0
    max_i = 0

    for i in range(tmp.shape[0]):
        line = tmp.loc[i, :]
        t = line['target']

        if t > max_t:
            max_t = t
            max_i = i

    max_line = tmp.loc[max_i, :]
    max_param = max_line['params']
    max_param['max_depth'] = round(max_param['max_depth'])
    max_param['num_leaves'] = round(max_param['num_leaves'])

    return max_param

In [24]:
no_bins = 8

df_month_9_10 = df_train[(df_train.startdate.dt.month == 9) | (df_train.startdate.dt.month == 10)]
# df_month_9_10 = df_train[((df_train.startdate >= '2015-08-15') & (df_train.startdate <= '2015-11-14')) | (df_train.startdate >= '2014-08-15') & (df_train.startdate <= '2014-11-14')]
df_month_9_10.loc[:, 'bin'] = pd.qcut(df_month_9_10[target_], q=no_bins, labels=range(no_bins))
df_month_9_10['final_target'] = df_month_9_10['idx'] + '_' + df_month_9_10['bin'].astype('str')
df_month_9_10.drop(columns=['idx', 'bin'], inplace=True)
df_month_9_10.head()

,index,lat,lon,startdate,contest-pevpr-sfc-gauss-14d__pevpr,nmme0-tmp2m-34w__cancm30,nmme0-tmp2m-34w__cancm40,nmme0-tmp2m-34w__ccsm30,nmme0-tmp2m-34w__ccsm40,nmme0-tmp2m-34w__cfsv20,...,nmme-prate-56w__cancm4_ewa,nmme-prate-56w__ccsm3_ewa,nmme-prate-56w__ccsm4_ewa,nmme-prate-56w__cfsv2_ewa,nmme-prate-56w__gfdl_ewa,nmme-prate-56w__gfdlflora_ewa,nmme-prate-56w__gfdlflorb_ewa,nmme-prate-56w__nasa_ewa,nmme-prate-56w__nmmemean_ewa,final_target
0,0,0.0,0.833333,2014-09-01,237.00,29.02,31.64,29.62,30.73,29.71,...,18.36,10.39,35.4,34.54,19.54,35.99,28.31,18.89,24.43,0.0_0.8333_7
1,1,0.0,0.833333,2014-09-02,228.90,29.02,31.64,29.62,30.73,29.71,...,18.36,10.39,35.4,34.54,19.54,35.99,28.31,18.89,24.43,0.0_0.8333_7
2,2,0.0,0.833333,2014-09-03,220.69,29.02,31.64,29.62,30.73,29.71,...,18.36,10.39,35.4,34.54,19.54,35.99,28.31,18.89,24.43,0.0_0.8333_7
3,3,0.0,0.833333,2014-09-04,225.28,29.02,31.64,29.62,30.73,29.71,...,18.36,10.39,35.4,34.54,19.54,35.99,28.31,18.89,24.43,0.0_0.8333_7
4,4,0.0,0.833333,2014-09-05,237.24,29.02,31.64,29.62,30.73,29.71,...,18.36,10.39,35.4,34.54,19.54,35.99,28.31,18.89,24.43,0.0_0.8333_7


In [25]:
df_train.drop(columns='idx', inplace=True)
df_test.drop(columns='idx', inplace=True)

In [26]:
import collections

print([item for item, count in collections.Counter(df_train.columns).items() if count > 1])

[]


In [27]:
dict_result = {}

In [28]:
from sklearn.model_selection import StratifiedKFold, KFold

X = df_month_9_10.copy()
y = df_month_9_10['final_target']

X.drop(columns='final_target', inplace=True)
df_month_9_10.drop(columns='final_target', inplace=True)

df_train = df_train[~((df_train.startdate.dt.month == 9) | (df_train.startdate.dt.month == 10))]
skf = StratifiedKFold(n_splits=5)

for i, (train_index, test_index) in enumerate(skf.split(X, y)):
    t = time.time()

    print("Training - {}".format(i))

    if i not in dict_result:
        print('Pre-processing...')

        X_train_fold = df_month_9_10.iloc[train_index]
        X_valid = df_month_9_10.iloc[test_index]

        X_train = pd.concat([df_train, X_train_fold])
        y_train = X_train[target_]
        y_valid = X_valid[target_]
        X_train.drop(columns=target_, inplace=True)
        X_valid.drop(columns=target_, inplace=True)

        test_index = df_test["index"].values
        X_test = df_test.copy()

        cat_cols = [i for i in X_train.select_dtypes(include='object').columns if i != date_col]
        X_train, X1, listEncoder = handle_feature_train_data(X_train, date_col, cat_cols.copy())
        X_valid = handle_feature_test_data(X_valid, listEncoder, columns_cat=cat_cols.copy())
        X_test = handle_feature_test_data(X_test, listEncoder, columns_cat=cat_cols.copy())

        drop_mei = [each for each in X_train.columns if "mei" in each]
        drop_ = [*drop_col, *["month", "day_of_year", "day_of_year_sin", "day_of_year_cos", "month_sin", "month_cos", "index"], *drop_mei, *col_drift]
        X_train = X_train.drop(columns=drop_)
        X_valid = X_valid.drop(columns=drop_)
        X_test = X_test.drop(columns=drop_)

        # X_train = X_train.drop(columns=target_)
        # X_valid = X_valid.drop(columns=target_)

        # Model
        print('Tuning...')

        # max_bo_params = bayes_tunning()
        # opt_params = get_bo_result(max_bo_params)

        opt_params = {
            'colsample_bytree': 0.7128556340156371,
            'feature_fraction_bynode': 0.7786840214424174,
            'max_depth': 13,
            'num_leaves': 14,
            'reg_alpha': 3.9,
            'reg_lambda': 1.665865282736549,
            'subsample': 0.6693424140753386,
            'metric': 'rmse',
            'n_estimators': 7999,
            'boosting_type': 'dart',
            'verbose': 0
        }

        reg = LGBMRegressor(**opt_params)

        reg.fit(X_train, y_train,
                eval_metric='rmse',
                categorical_feature=[
                    *['climateregions__climateregion', 'idx', 'mjo1d__phase'],
                    *[each + "_bin" for each in bin_col]
                ])

        ypred_train = reg.predict(X_train)
        ypred_valid = reg.predict(X_valid)

        result_train = mean_squared_error(y_train, ypred_train, squared=False)
        result_valid = mean_squared_error(y_valid, ypred_valid, squared=False)

        print("Take time: ", time.time() - t)
        print("Train_score: {}  Valid_score: {}".format(result_train, result_valid))

        ypred_test = reg.predict(X_test)
        dict_result[i] = ypred_test

    else:
        print('Fold was trained!')

    print("-------------")

Training - 0
Pre-processing...
Tuning...
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.901622 seconds.
You can set `force_col_wise=true` to remove the overhead.


KeyboardInterrupt: 

In [ ]:
ypred_test = np.mean([value for key, value in dict_result.items()], axis=0)

In [ ]:
pd.DataFrame(data = {"{}".format(target_): ypred_test, "index": test_index}).to_csv("submit_lgbm_bo.csv", index=False)

In [ ]:
X_train.shape

In [ ]:
importances = reg.feature_importances_

data = {'col': X_train.columns, 'imp': importances}

ft_imp_df = pd.DataFrame(data)
ft_imp_df.sort_values('imp', ascending=False)

In [ ]:
ft_imp_df[ft_imp_df['col'].str.contains('climateregion_')].sort_values('imp', ascending=False)

In [ ]:
len(set(X_train.columns))

In [ ]:
ft_imp_df.to_csv('ft_imp.csv')

# Pseudo Labeling and Postprocessing

In [ ]:
# Pseudo Labelling
train_pseudo = X_test.copy()
ddf = pd.read_csv('submit_res.csv')
y_test_pred  = ddf[target_].values.flatten()

train_pseudo[target_] = y_test_pred

X_train_pseudo = pd.concat([X_train, X_valid]).copy()
X_train_pseudo[target_] = [*y_train, *y_valid]

train_mod = pd.concat([X_train_pseudo.copy(), train_pseudo], axis=0).reset_index(drop=True)

features = [c for c in X_test.columns if (c != 'index')]
display(train_mod)

XX = train_mod[features]
yy = train_mod[target_]
y_oof_pred = np.zeros(len(yy))

X_testt = X_test[features].values
y_test_pred2 = np.zeros(len(X_testt))

In [49]:
from sklearn.model_selection import train_test_split

yy[np.isnan(yy)] = 0

X_train3, X_test3, y_train3, y_test3 = train_test_split(XX, yy, test_size=0.3, random_state=42)

for each in cat_cols:
    X_train3[each] = X_train3[each].astype('category')
    X_test3[each] = X_test3[each].astype('category')
    X_test[each] = X_test[each].astype('category')

bst2 = LGBMRegressor(**opt_params)
bst2.fit(X_train3, y_train3, eval_set=(X_test3, y_test3),
         eval_metric='rmse',
         categorical_feature=[
            *['climateregions__climateregion', 'idx', 'mjo1d__phase'],
            *[each + "_bin" for each in bin_col]
         ])

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.884761 seconds.
You can set `force_col_wise=true` to remove the overhead.
[1]	valid_0's rmse: 8.78708
[2]	valid_0's rmse: 8.00315
[3]	valid_0's rmse: 7.30364
[4]	valid_0's rmse: 6.67998
[5]	valid_0's rmse: 6.11801
[6]	valid_0's rmse: 5.62056
[7]	valid_0's rmse: 5.17679
[8]	valid_0's rmse: 5.41044
[9]	valid_0's rmse: 4.98684
[10]	valid_0's rmse: 4.61035
[11]	valid_0's rmse: 4.27721
[12]	valid_0's rmse: 4.43179
[13]	valid_0's rmse: 4.11704
[14]	valid_0's rmse: 3.82906
[15]	valid_0's rmse: 3.58026
[16]	valid_0's rmse: 3.35765
[17]	valid_0's rmse: 3.15049
[18]	valid_0's rmse: 2.96605
[19]	valid_0's rmse: 2.80851
[20]	valid_0's rmse: 2.65839
[21]	valid_0's rmse: 2.74161
[22]	valid_0's rmse: 2.59683
[23]	valid_0's rmse: 2.4684
[24]	valid_0's rmse: 2.35548
[25]	valid_0's rmse: 2.24803
[26]	valid_0's rmse: 2.15506
[27]	valid_0's rmse: 2.073
[28]	valid_0's rmse: 2.1259
[29]	valid_0's rmse: 2.04291
[30]	va

LGBMRegressor(boosting_type='dart', colsample_bytree=0.7128556340156371,
              feature_fraction_bynode=0.7786840214424174, max_depth=13,
              metric='rmse', n_estimators=7999, num_leaves=14, reg_alpha=3.9,
              reg_lambda=1.665865282736549, subsample=0.6693424140753386,
              verbose=0)

In [50]:
y_pseudo_pred = bst2.predict(X_test)
pd.DataFrame(data = {"{}".format(target_): y_pseudo_pred, "index": test_index}).to_csv("submit_lgbm_pseudo.csv", index=False)

# Ensemble

In [292]:
submit_tunning_1 = pd.read_csv('submit_lgbm.csv')

submit_lgbm = pd.read_csv('submit_lgbm_bo.csv')

submit_res = pd.read_csv('submit_res.csv')

submit_lgbm.head()

,contest-tmp2m-14d__tmp2m,index
0,28.739887,375734
1,28.698812,375735
2,28.734199,375736
3,28.808007,375737
4,28.838081,375738


In [293]:
submit_ensemple = pd.DataFrame()

submit_ensemple[target_] = (submit_lgbm[target_] + submit_tunning_1[target_] + submit_res[target_]) / 3
submit_ensemple['index'] = submit_lgbm['index']

submit_ensemple.to_csv('submit_ensemble.csv', index=False)

In [297]:
submit_res.tail(50)

,contest-tmp2m-14d__tmp2m,index
31304,13.672541,407038
31305,13.514550,407039
31306,13.787864,407040
31307,13.753263,407041
31308,13.521520,407042
31309,13.445884,407043
31310,13.556410,407044
31311,13.210343,407045
31312,13.428071,407046
31313,13.390244,407047


In [298]:
submit_ensemple.tail(50)

,contest-tmp2m-14d__tmp2m,index
31304,13.671254,407038
31305,13.526216,407039
31306,13.751920,407040
31307,13.707600,407041
31308,13.472247,407042
31309,13.385757,407043
31310,13.520935,407044
31311,13.129234,407045
31312,13.351761,407046
31313,13.348171,407047


## Linear Regression

In [ ]:
class TargetEncoder(base.BaseEstimator,base.TransformerMixin):
    def __init__(self, colnames, targetName, colSupport=None,
                  discardOriginal_col=False):
        self.colnames = colnames
        self.colSupport = colSupport
        self.targetName = targetName
        self.discardOriginal_col = discardOriginal_col
        self.new_cols = colnames + '_tar'
        self.data_transform = None

    def fit(self, X, y=None):
        if self.colSupport != None:
            self.data_transform = X.groupby([self.colnames, self.colSupport]).agg({self.targetName: 'mean'}).rename(columns={self.targetName: self.new_cols})
        else:
            self.data_transform = X.groupby([self.colnames]).agg({self.targetName: 'mean'}).rename(columns={self.targetName: self.new_cols})
        return self

    def transform(self, X,train=True):
        X = X.merge(self.data_transform.reset_index())
        encoded_feature = X[self.new_cols].values
        if train:
            print('Correlation between the new feature, {} and, {} is {}.'.format(self.new_cols,self.targetName,
                   ma.corrcoef(ma.masked_invalid(X[self.targetName].values), ma.masked_invalid(encoded_feature))[0][1]))
        if self.discardOriginal_col:
            X = X.drop(self.targetName, axis=1)
        return X

In [ ]:
dict_result = {}
year_valid = [2015, 2015, 2015, 2015, 2015, 2015, 2016, 2016, 2016, 2016]
month_valid = [1, 3, 5, 7, 9, 11, 1, 3, 5, 7]
gap = 1

for index in range(len(year_valid)):
    if index in dict_result:
        continue
    y, m_s, m_e = year_valid[index], month_valid[index], month_valid[index] + 1
    X_valid = df_train[(df_train[date_col].dt.year == y) & (df_train[date_col].dt.month >= m_s) & (df_train[date_col].dt.month <= m_e)]
    X_train = df_train[~((df_train[date_col].dt.year == y) & (df_train[date_col].dt.month >= m_s) & (df_train[date_col].dt.month <= m_e))]

    # y_train = X_train[target_].copy()
    X_train = X_train.drop(columns=["index"])
    # y_valid = X_valid[target_].copy()
    X_valid = X_valid.drop(columns=["index"])

    test_index = df_test["index"].values
    X_test = df_test.drop(columns=["index"]).copy()

    cat_cols = [i for i in X_train.select_dtypes(include='object').columns if i != date_col]
    X_train, listEncoder, df_mean = handle_feature_train_data(X_train, date_col, cat_cols.copy())
    X_valid = handle_feature_test_data(X_valid, listEncoder, df_mean, columns_cat=cat_cols.copy())
    X_test = handle_feature_test_data(X_test, listEncoder, df_mean, columns_cat=cat_cols.copy())

    target_climate = TargetEncoder(colnames='climateregions__climateregion', targetName=target_).fit(X_train)
    target_idx = TargetEncoder(colnames='idx', targetName=target_).fit(X_train)

    X_train = target_climate.transform(X_train)
    X_valid = target_climate.transform(X_valid)
    X_test = target_climate.transform(X_test, train=False)
    X_train = target_idx.transform(X_train)
    X_valid = target_idx.transform(X_valid)
    X_test = target_idx.transform(X_test, train=False)

    y_train = X_train[target_].copy()
    y_valid = X_valid[target_].copy()
    X_train = X_train.drop(columns=[target_])
    X_valid = X_valid.drop(columns=[target_])

    drop_ = [*drop_col, *["mei__nip", "idx", "climateregions__climateregion", 'month', 'year'], *[col for col in X_train.columns if 'Unnamed' in col]]
    # drop_ = ["month", "day_of_year", "day_of_year_sin", "day_of_year_cos", "month_sin", "month_cos", "mei__nip"]

    X_train_ = X_train.drop(columns=drop_)
    X_valid_ = X_valid.drop(columns=drop_)
    X_test_ = X_test.drop(columns=drop_)

    print("Training - {}".format(index))

    t = time.time()
    lgb = LinearRegression(normalize=True)
    lgb.fit(X_train_, y_train)
    print("Take time: ", time.time() - t)

    ypred_test = lgb.predict(X_test_)
    dict_result[index] = ypred_test

    result_valid_ = mean_squared_error(y_valid, lgb.predict(X_valid_), squared=False)
    print(result_valid_)
    print('-----------------')

In [ ]:
ypred_test = np.mean([value for key, value in dict_result.items()], axis=0)
pd.DataFrame(data = {"{}".format(target_): ypred_test, "index": test_index}).to_csv("submit_linear.csv", index=False)

## Ridge

In [ ]:
dict_result1 = {}

year_valid = [2015, 2015, 2015, 2015, 2015, 2015, 2016, 2016, 2016, 2016]
month_valid = [1, 3, 5, 7, 9, 11, 1, 3, 5, 7]

for index in range(len(year_valid)):
    t = time.time()

    if index in dict_result1:
        continue
    y, m_s, m_e = year_valid[index], month_valid[index], month_valid[index] + 1

    print("Training - {}".format(index))
    print(f'Test on month {m_s} and {m_e}, year {y}')

    X_valid = df_train[(df_train[date_col].dt.year == y) & (df_train[date_col].dt.month >= m_s) & (df_train[date_col].dt.month <= m_e)]
    X_train = df_train[~((df_train[date_col].dt.year == y) & (df_train[date_col].dt.month >= m_s) & (df_train[date_col].dt.month <= m_e))]

    y_train = X_train[target_]
    y_valid = X_valid[target_]

    date = np.array(X_valid['startdate'])

    test_index = df_test["index"].values
    X_test = df_test.copy()

    cat_cols = [i for i in X_train.select_dtypes(include='object').columns if i != date_col]
    X_train, listEncoder, df_mean = handle_feature_train_data(X_train, date_col, cat_cols.copy())
    X_valid = handle_feature_test_data(X_valid, listEncoder, df_mean, columns_cat=cat_cols.copy())
    X_test = handle_feature_test_data(X_test, listEncoder, df_mean, columns_cat=cat_cols.copy())

    drop_ = [*drop_col, *[col for col in X_train.columns if 'Unnamed' in col]]
    # drop_ = [col for col in X_train.columns if 'Unnamed' in col]
    X_train = X_train.drop(columns=drop_)
    X_valid = X_valid.drop(columns=drop_)
    X_test = X_test.drop(columns=drop_)

    X_train = X_train.drop(columns=target_)
    X_valid = X_valid.drop(columns=target_)

    reg1 = Ridge(alpha=1)
    reg1.fit(X_train, y_train)

    ypred_train1 = reg1.predict(X_train)
    ypred_valid1 = reg1.predict(X_valid)

    result_train = mean_squared_error(y_train, ypred_train1, squared=False)
    result_valid = mean_squared_error(y_valid, ypred_valid1, squared=False)

    ypred_test1 = reg1.predict(X_test)
    dict_result1[index] = ypred_test1

    print("Take time: ", time.time() - t)
    print("Train_score: {}  Valid_score: {}".format(result_train, result_valid))
    print("-------------")

In [ ]:
ypred_test = np.mean([value for key, value in dict_result.items()], axis=0)
pd.DataFrame(data = {"{}".format(target_): ypred_test, "index": test_index}).to_csv("submit_ridge.csv", index=False)

# Ensemble